In [1]:
import socket
import struct
import json
VALIDACAO = '2019057000:21122021:fd41751065ed3b869ef705d4f4b56ee0a853659a4ee2ec3999d63f4a257368ad+2019057001:21122021:96bf30b7c93eea60e3e7c6783e840a64d51a236685b06a3561f57931e8b54227+5eb1322e40a8dcf937ef041265c5bf00b7f23f363590c771db6f5414ebcdbb1a'

def entrada():
    while True:
        entrada = input('Digite sua entrada: ')
        dados = entrada.split(' ')
        try:
            s = dados[0]
            p = int(dados[1])
            n = VALIDACAO
            break
        except:
            print('Entrada inválida! Tente no seguinte formato: "SERVER PORTA SAG"')
            
    while s != VALID_SERVER:
        entrada = input('Server inválidado! Digite novamente: ')
        dados = entrada.split(' ')
        s = dados[0]
        p = int(dados[1])
        n = VALIDACAO
        
    while p not in VALID_PORTS:
        entrada = input('Porta inválidada! Digite novamente: ')
        dados = entrada.split(' ')
        s = dados[0]
        p = int(dados[1])
        n = VALIDACAO

    info =[s,p,n]
    return info

def authreq(rio, adress):
    #ENVIO
    entrada = json.dumps({"type": "authreq", "auth": SAG}).encode('utf-8')
    rio.sendto(entrada, adress)
        
    #RESPOSTA
    try:
        saida = rio.recv(bufferSize, 0)
        resposta = json.loads(saida.decode('utf-8'))
        print(resposta)
        ESTADO.append(resposta['type'])
        if 'gameover' in ESTADO:
                quit()
        return 0
    except:
        print('Erro de transmissão')
        return 1

def getcannons():
    #ENVIO
    entrada = json.dumps({"type": "getcannons", "auth": SAG}).encode('utf-8')
    rio1.sendto(entrada, RIVER[0]) 

    #RESPOSTA
    try:
        saida = rio1.recv(bufferSize, 0)
        resposta = json.loads(saida.decode('utf-8'))
        print(resposta)
        VALID_CANNONS.append(resposta['cannons'])
        ESTADO.append(resposta['type'])
    except:
        print('Erro de transmissão')
        getcannons()

def getturn(turn, rio, adress):
    #ENVIO
    entrada = json.dumps({"type": "getturn", "auth": SAG, "turn": turn}).encode('utf-8')
    rio.sendto(entrada, adress)
    
    
def state(rio, boat, lista_resposta):
        try:
            for t in range(0,8):
                saida = rio.recv(bufferSize, 0)
                resposta = json.loads(saida.decode('utf-8'))
                tam = len(resposta['ships'])
                for c in range(0,tam):
                    ponte = resposta['bridge']
                    boat[ponte-1].append(resposta['ships'][c]['id'])
                    ALL_BOATS.append(resposta['ships'][c]['id'])
                ESTADO.append(resposta['type'])
                lista_resposta[t] = resposta
            return 0
        except:
            print('erro de transmissão no state', flush=True)
            return 1
            

def shot(rio, adress, cannon, id):
    entrada = json.dumps({"type": "shot", "auth": SAG, "cannon": cannon, "id": id}).encode('utf-8')
    rio.sendto(entrada, adress)

    #RESPOSTA
    try:
        saida = rio.recv(bufferSize, 0)
        resposta = json.loads(saida.decode('utf-8'))
        print(resposta)
        #ESTADO.append(resposta['type'])
    except:
        #print('Erro de transmissão')
        shot(rio, adress, cannon, id)

            
def quit():
    #ENVIO
    entrada = json.dumps({"type": "quit", "auth": SAG}).encode('utf-8')                          
    rio1.sendto(entrada, RIVER[0])
    
    rio1.close()
    rio2.close()
    rio3.close()
    rio4.close()
    print('Jogo finalizado com sucesso!')
    exit()

    #bd20212.dcc023.2advanced.dev 52221

#DEFININDO AS ESPECIFICAÇÕES DO SERVIDOR E PEGANDO AS INFORMAÇÕES DO TECLADO
bufferSize = 4096
RIVER = [0,0,0,0]
VALID_PORTS = [52221,52222,52223,52224]
VALID_SERVER = 'bd20212.dcc023.2advanced.dev'
VALID_CANNONS = []

ESTADO = []
RIOS = [1,2,3,4]
timeout = 0.5

rio1 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
rio2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
rio3 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
rio4 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
auth = list()

for c in range(0,4):
    dados = entrada()
    SERVER = dados[0]
    PORT = int(dados[1])
    SAG = dados[2]
    
    if PORT == VALID_PORTS[0]:
        RIVER[0]=(SERVER, PORT)
        rio1.connect(RIVER[0])
        rio1.settimeout(timeout) 
    elif PORT == VALID_PORTS[1]:
        RIVER[1]=(SERVER, PORT)
        rio2.connect(RIVER[1])
        rio2.settimeout(timeout) 
    elif PORT == VALID_PORTS[2]:
        RIVER[2]=(SERVER, PORT)
        rio3.connect(RIVER[2])
        rio3.settimeout(timeout)
    elif PORT == VALID_PORTS[3]:
        RIVER[3]=(SERVER, PORT)
        rio4.connect(RIVER[3])
        rio4.settimeout(timeout)


print('INICIANDO O JOGO: ')
auth.append(authreq(rio1, RIVER[0]))
while auth[0] == 1:
    auth[0] = (authreq(rio1, RIVER[0]))

auth.append(authreq(rio2, RIVER[1]))
while auth[1] == 1:
    auth[1] = (authreq(rio2, RIVER[1]))

auth.append(authreq(rio3, RIVER[2]))
while auth[2] == 1:
    auth[2] = (authreq(rio3, RIVER[2]))
    
auth.append(authreq(rio4, RIVER[3]))
while auth[3] == 1:
    auth[3] = (authreq(rio4, RIVER[3]))

if auth == [0,0,0,0]:
    print('AUTENTICAÇÃO COMPLETA')
    getcannons()
    turno = 0

    while turno < 274:
        if turno < 250:
            RESPOSTA_RIO1 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO2 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO3 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO4 = ['', '', '', '', '', '', '', '']
            ALL_BOATS = []
            BOATS_1 = [[],[],[],[],[],[],[],[]]
            BOATS_2 = [[],[],[],[],[],[],[],[]]
            BOATS_3 = [[],[],[],[],[],[],[],[]]
            BOATS_4 = [[],[],[],[],[],[],[],[]]
            error = [0,0,0,0]
            print(f'TURNO {turno}')

            getturn(turno, rio1, RIVER[0])
            getturn(turno, rio2, RIVER[1])
            getturn(turno, rio3, RIVER[2])
            getturn(turno, rio4, RIVER[3])

            error[0] = state(rio1, BOATS_1, RESPOSTA_RIO1)
            if error[0] == 1:
                for c in range(0,4):
                    getturn(turno, rio1, RIVER[0])
                    error[0] = state(rio1, BOATS_1, RESPOSTA_RIO1)
                    if error[0] == 0:
                        break

            error[1] = state(rio2, BOATS_2, RESPOSTA_RIO2)
            if error[1] == 1:
                for c in range (0,4):
                    getturn(turno, rio2, RIVER[1])
                    error[1] = state(rio2, BOATS_2, RESPOSTA_RIO2)
                    if error[1]==0:
                        break
            
            error[2] = state(rio3, BOATS_3, RESPOSTA_RIO3)
            if error[2] == 1:
                for c in range(0,4):
                    getturn(turno, rio3, RIVER[2])
                    error[2] = state(rio3, BOATS_3, RESPOSTA_RIO3)
                    if error[2]==0:
                        break
            
            error[3] = state(rio4, BOATS_4, RESPOSTA_RIO4)
            if  error[3] == 1:
                for c in range(0,4):
                    getturn(turno, rio4, RIVER[3])
                    error[3] = state(rio4, BOATS_4, RESPOSTA_RIO4)
                    if error[3]==0:
                        break

            print('\nRIO 1:')
            for c in RESPOSTA_RIO1:
                print(c)

            print('\nRIO 2:')
            for c in RESPOSTA_RIO2:
                print(c)

            print('\nRIO 3:')
            for c in RESPOSTA_RIO3:
                print(c)

            print('\nRIO 4:')
            for c in RESPOSTA_RIO4:
                print(c)

            print('NAVIOS DISPONIVEIS:', end=' ')
            print(ALL_BOATS)
            print ("RIO 1: ", BOATS_1)
            print ("RIO 2: ", BOATS_2)
            print ("RIO 3: ", BOATS_3)
            print ("RIO 4: ", BOATS_4)

            try:
                for x in VALID_CANNONS[0]:
                    p = x[0]
                        
                    if x[1] == 0:
                        r = 1
                        if len(BOATS_1[p-1]) > 0:
                            identificador = BOATS_1[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]

                    if x[1] == 1:
                        r = 1
                        if len(BOATS_1[p-1]) > 0:
                            identificador = BOATS_1[p-1][0]
                        elif len(BOATS_2[p-1]) > 0:
                            r = 2
                            identificador = BOATS_2[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 2:
                        r = 2
                        if len(BOATS_2[p-1]) > 0:
                            identificador = BOATS_2[p-1][0]
                        elif len(BOATS_3[p-1]) > 0:
                            r = 3
                            identificador = BOATS_3[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 3:
                        r = 3
                        if len(BOATS_3[p-1]) > 0:
                            identificador = BOATS_3[p-1][0]
                        elif len(BOATS_4[p-1]) > 0:
                            r = 4
                            identificador = BOATS_4[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 4:
                        r = 4
                        if len(BOATS_4[p-1]) > 0:
                            identificador = BOATS_4[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
            
                    if r == 1:
                        shot(rio1, RIVER[0], x, identificador)
                    if r == 2:
                        shot(rio2, RIVER[1], x, identificador)
                    if r == 3:
                        shot(rio3, RIVER[2], x, identificador)
                    if r == 4:
                        shot(rio4, RIVER[3], x, identificador)
                
            except:
                print('erro ao atirar')
                
            turno += 1
            #input('proximo round')
            if 'gameover' in ESTADO:
                break
        
        else:
            RESPOSTA_RIO1 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO2 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO3 = ['', '', '', '', '', '', '', '']
            RESPOSTA_RIO4 = ['', '', '', '', '', '', '', '']
            ALL_BOATS = []
            BOATS_1 = [[],[],[],[],[],[],[],[]]
            BOATS_2 = [[],[],[],[],[],[],[],[]]
            BOATS_3 = [[],[],[],[],[],[],[],[]]
            BOATS_4 = [[],[],[],[],[],[],[],[]]
            error = [0,0,0,0]
            print(f'TURNO {turno}')

            getturn(turno, rio1, RIVER[0])
            getturn(turno, rio2, RIVER[1])
            getturn(turno, rio3, RIVER[2])
            getturn(turno, rio4, RIVER[3])

            error[0] = state(rio1, BOATS_1, RESPOSTA_RIO1)
            error[1] = state(rio2, BOATS_2, RESPOSTA_RIO2)
            error[2] = state(rio3, BOATS_3, RESPOSTA_RIO3)
            error[3] = state(rio4, BOATS_4, RESPOSTA_RIO4)
            print('\nRIO 1:')
            for c in RESPOSTA_RIO1:
                print(c)

            print('\nRIO 2:')
            for c in RESPOSTA_RIO2:
                print(c)

            print('\nRIO 3:')
            for c in RESPOSTA_RIO3:
                print(c)

            print('\nRIO 4:')
            for c in RESPOSTA_RIO4:
                print(c)

            print('NAVIOS DISPONIVEIS:', end=' ')
            print(ALL_BOATS)
            print ("RIO 1: ", BOATS_1)
            print ("RIO 2: ", BOATS_2)
            print ("RIO 3: ", BOATS_3)
            print ("RIO 4: ", BOATS_4)

            try:
                for x in VALID_CANNONS[0]:
                    p = x[0]
                        
                    if x[1] == 0:
                        r = 1
                        if len(BOATS_1[p-1]) > 0:
                            identificador = BOATS_1[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]

                    if x[1] == 1:
                        r = 1
                        if len(BOATS_1[p-1]) > 0:
                            identificador = BOATS_1[p-1][0]
                        elif len(BOATS_2[p-1]) > 0:
                            r = 2
                            identificador = BOATS_2[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 2:
                        r = 2
                        if len(BOATS_2[p-1]) > 0:
                            identificador = BOATS_2[p-1][0]
                        elif len(BOATS_3[p-1]) > 0:
                            r = 3
                            identificador = BOATS_3[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 3:
                        r = 3
                        if len(BOATS_3[p-1]) > 0:
                            identificador = BOATS_3[p-1][0]
                        elif len(BOATS_4[p-1]) > 0:
                            r = 4
                            identificador = BOATS_4[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
                    if x[1] == 4:
                        r = 4
                        if len(BOATS_4[p-1]) > 0:
                            identificador = BOATS_4[p-1][0]
                        else:
                            identificador = ALL_BOATS[0]
                        
            
                    if r == 1:
                        shot(rio1, RIVER[0], x, identificador)
                    if r == 2:
                        shot(rio2, RIVER[1], x, identificador)
                    if r == 3:
                        shot(rio3, RIVER[2], x, identificador)
                    if r == 4:
                        shot(rio4, RIVER[3], x, identificador)
                
            except:
                print('erro ao atirar')
                
            turno += 1
            #input('proximo round')
            if 'gameover' in ESTADO:
                break

else:
    print('FALHA NA AUTENTICAÇÃO')

print(f'ESTADO {ESTADO}')
print('Gameover!')
quit()

INICIANDO O JOGO: 
{'type': 'authresp', 'status': 0, 'river': 1, 'auth': '2019057000:21122021:fd41751065ed3b869ef705d4f4b56ee0a853659a4ee2ec3999d63f4a257368ad+2019057001:21122021:96bf30b7c93eea60e3e7c6783e840a64d51a236685b06a3561f57931e8b54227+5eb1322e40a8dcf937ef041265c5bf00b7f23f363590c771db6f5414ebcdbb1a'}
{'type': 'authresp', 'status': 0, 'river': 2, 'auth': '2019057000:21122021:fd41751065ed3b869ef705d4f4b56ee0a853659a4ee2ec3999d63f4a257368ad+2019057001:21122021:96bf30b7c93eea60e3e7c6783e840a64d51a236685b06a3561f57931e8b54227+5eb1322e40a8dcf937ef041265c5bf00b7f23f363590c771db6f5414ebcdbb1a'}
{'type': 'authresp', 'status': 0, 'river': 3, 'auth': '2019057000:21122021:fd41751065ed3b869ef705d4f4b56ee0a853659a4ee2ec3999d63f4a257368ad+2019057001:21122021:96bf30b7c93eea60e3e7c6783e840a64d51a236685b06a3561f57931e8b54227+5eb1322e40a8dcf937ef041265c5bf00b7f23f363590c771db6f5414ebcdbb1a'}
{'type': 'authresp', 'status': 0, 'river': 4, 'auth': '2019057000:21122021:fd41751065ed3b869ef705d4f4b56e

### DESAFIOS ENCONTRADOS: 
- Descobrir como passar um JSON para o servidor
- Descobrir quando ocorria o erro de transmissão de dados do servidor e como corrigi-lo
- Corrigir possiveis entradas erradas do cliente